In [1]:
import numpy as np
import pandas as pd
import uproot
import pyarrow.parquet as pq
import awkward_pandas as akpd
import itertools
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import matplotlib.pyplot as plt
import time

In [5]:
if True:
    path = '/groups/hep/kinch/H_Zg/samples_processed/HZmmg_ggF_MC_reduced_11sep_157var.parquet'
else:
    path = 'c:/Users/Jens/Documents/Github/Thesis/samples_processed/HZeeg_ggF_MC_reduced_28august_131var.parquet'

sample = pd.read_parquet(path) 
print(sample.shape)

(80000, 131)


In [3]:
sample_small = sample.head(100)
print(sample_small.shape)

(100, 131)


In [4]:
headers = sample.columns
print("Headers:")
for header in headers:
    # print(header)
    print(f'{header}, {sample[header].iloc[108]}, {type(sample[header].iloc[108])}')
    # first_index_shape = sample[header].iloc[0].shape if isinstance(sample[header].iloc[0], (pd.Series, np.ndarray)) else None
    #print("Shape of the first index:", first_index_shape)


Headers:
runNumber, 410000, <class 'numpy.uint32'>
eventNumber, 106246, <class 'numpy.uint64'>
mcEventNumber, 106246, <class 'numpy.uint64'>
actualInteractionsPerCrossing, 49.5, <class 'numpy.float32'>
averageInteractionsPerCrossing, 49.5, <class 'numpy.float32'>
mcEventWeights, [31, 31, 19.5, 29.4, 29, 19.8, 32.4, ..., 27.7, 29.9, 31, 31, 31.6, 30.4, -310], <class 'awkward.highlevel.Array'>
truthel_m, [0.511, 0.511], <class 'awkward.highlevel.Array'>
truthel_px, [-52, -108], <class 'awkward.highlevel.Array'>
truthel_py, [6.92, 13.4], <class 'awkward.highlevel.Array'>
truthel_pz, [-26.6, -51.7], <class 'awkward.highlevel.Array'>
truthel_E, [58.8, 120], <class 'awkward.highlevel.Array'>
truthel_pdgId, [11, -11], <class 'awkward.highlevel.Array'>
truthel_ptcone30, [-1, -1], <class 'awkward.highlevel.Array'>
truthel_etcone20, [-1, -1], <class 'awkward.highlevel.Array'>
truthel_ParticleOrigin, [6, 6], <class 'awkward.highlevel.Array'>
truthel_Classification, [132, 132], <class 'awkward.hig

In [ ]:
def GSF_Track_finder(row):
    num_electrons = len(row["mu_truthOrigin"])
    dR_min = np.ones(num_electrons)*999
    GSF_index = np.ones(num_electrons,dtype=int)*-1
    for i_el in range(num_electrons):
        NGSF = len(row["GSFTrackParticlesAuxDyn_theta"])
        for i_GSF in range(NGSF):
            eta_GSF = -np.log(np.tan(row['GSFTrackParticlesAuxDyn_theta'][i_GSF]/2))
            deta = row['el_eta'][i_el] - eta_GSF
            dphi = row['el_phi'][i_el] - row['GSFTrackParticlesAuxDyn_phi'][i_GSF]
            if (dphi > np.pi) : dphi = dphi - np.pi
            if (dphi < -np.pi) : dphi = dphi + np.pi
            dR_eG = np.sqrt(deta**2 + dphi**2)
            if (dR_eG < dR_min[i_el]):
                dR_min[i_el] = dR_eG
                GSF_index[i_el] = i_GSF
    return GSF_index, dR_min

In [ ]:
sample_small = sample

In [ ]:
# Create new dataframe containing all the pair of electrons
# Create an empty list to store the new rows
new_rows = []

# Iterate through each row of the original DataFrame
# for index, row in sample.iterrows():

time2_array = []
start_loop = time.time()

